In [1]:
import torch
import torchvision
import math
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm_notebook
from torch.utils.tensorboard import SummaryWriter


from GWD_data import WheatDataset
from metric import calculate_image_precision
from utils import collate_fn, plot_boxes, get_model_name, format_prediction_string, remove_blanks, is_contain_blanks
from config import config


In [2]:
conf = config()
conf.DEVICE = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
conf.BATH_SIZE = 4
torch.random.manual_seed(5)
writer = SummaryWriter("runs/Validate") #


WD_Train = WheatDataset(conf)
WD_Valid = WheatDataset(conf, train=False)
WD_Train_Loader = torch.utils.data.DataLoader(WD_Train, batch_size=conf.BATH_SIZE, shuffle=True, collate_fn=collate_fn)
WD_Valid_Loader = torch.utils.data.DataLoader(WD_Valid, batch_size=conf.BATH_SIZE, shuffle=True, collate_fn=collate_fn)

In [3]:
import os
os.listdir('model/')

['global_wheat_epoch_0_loss_0.7703235149383545.model',
 'global_wheat_epoch_0_step_0_loss_1.7423330545425415.model',
 'global_wheat_epoch_0_step_0_loss_1.792172908782959.model',
 'global_wheat_epoch_0_step_100_loss_1.3611688613891602.model',
 'global_wheat_epoch_0_step_200_loss_1.02377450466156.model',
 'global_wheat_epoch_0_step_300_loss_0.8959460258483887.model',
 'global_wheat_epoch_0_step_400_loss_0.8438630104064941.model',
 'global_wheat_epoch_4_loss_0.7505825161933899.model',
 'GWD_EPOCH_0_SCORE_0.0000_LOSS_382.9978.pt',
 'GWD_EPOCH_10_SCORE_0.0000_LOSS_211.9845.pt',
 'GWD_EPOCH_11_SCORE_0.0000_LOSS_209.1651.pt',
 'GWD_EPOCH_12_SCORE_0.0000_LOSS_208.6549.pt',
 'GWD_EPOCH_13_SCORE_0.0000_LOSS_206.3007.pt',
 'GWD_EPOCH_14_SCORE_0.0000_LOSS_205.7413.pt',
 'GWD_EPOCH_15_SCORE_0.0000_LOSS_205.9956.pt',
 'GWD_EPOCH_16_SCORE_0.0000_LOSS_203.7507.pt',
 'GWD_EPOCH_17_SCORE_0.0000_LOSS_201.9021.pt',
 'GWD_EPOCH_18_SCORE_0.0000_LOSS_203.8619.pt',
 'GWD_EPOCH_19_SCORE_0.0000_LOSS_200.8293.pt

In [4]:
model_list = [
 'GWD_EPOCH_16_SCORE_0.0000_LOSS_203.7507.pt',
 'GWD_EPOCH_17_SCORE_0.0000_LOSS_201.9021.pt',
 'GWD_EPOCH_18_SCORE_0.0000_LOSS_203.8619.pt',
 'GWD_EPOCH_19_SCORE_0.0000_LOSS_200.8293.pt',
 'GWD_EPOCH_20_SCORE_0.0000_LOSS_197.1215.pt',
 'GWD_EPOCH_21_SCORE_0.0000_LOSS_195.4317.pt',
 'GWD_EPOCH_22_SCORE_0.0000_LOSS_194.9353.pt',
 'GWD_EPOCH_23_SCORE_0.0000_LOSS_193.9966.pt',
 'GWD_EPOCH_24_SCORE_0.0000_LOSS_193.6854.pt',
 'GWD_EPOCH_25_SCORE_0.0000_LOSS_194.8778.pt',
 'GWD_EPOCH_26_SCORE_0.0000_LOSS_193.3428.pt',
 'GWD_EPOCH_27_SCORE_0.0000_LOSS_194.3706.pt',
 'GWD_EPOCH_28_SCORE_0.0000_LOSS_192.9131.pt',
 'GWD_EPOCH_29_SCORE_0.0000_LOSS_191.8370.pt',
 'GWD_EPOCH_30_SCORE_0.0000_LOSS_190.1350.pt',
 'GWD_EPOCH_31_SCORE_0.0000_LOSS_191.7623.pt',
 'GWD_EPOCH_32_SCORE_0.0000_LOSS_189.1150.pt',
 'GWD_EPOCH_33_SCORE_0.0000_LOSS_189.5725.pt',
 'GWD_EPOCH_34_SCORE_0.0000_LOSS_188.9933.pt',
 'GWD_EPOCH_35_SCORE_0.0000_LOSS_190.0600.pt']

# Testing Data

In [5]:
torch.cuda.empty_cache()

In [6]:
for i in tqdm_notebook(range(len(model_list))):
    
    EPOCH = int(model_list[i].split('_')[2])
    LOSS = float(model_list[i].split('_')[-1][:-3])
    
    GWD_Model = torch.load('model/'+model_list[i])
    GWD_Model.eval()
    GWD_Model.to(conf.DEVICE)
    precisions = []
    
    for images, targets in tqdm_notebook(WD_Valid_Loader):
        try:
            images = [torch.tensor(image, dtype = torch.float32).to(conf.DEVICE) for image in images]
            preds = GWD_Model(images)

            for pred, target in zip(preds, targets):
                ground_truth = target['boxes']
                pred_boxes = pred['boxes'].to(torch.device('cpu')).detach().numpy()
                precisions.append(calculate_image_precision(ground_truth, pred_boxes, thresholds=conf.PRECISION_THRESH))
        except:
            print('weird shape error')
    
    test_score = np.mean(np.array(precisions))
    writer.add_scalar('Epoch_Vars/Model_Loss', LOSS, EPOCH)
    writer.add_scalar('Epoch_Vars/Validation_Score', test_score, EPOCH)
    
    GWD_Model=None
    torch.cuda.empty_cache()
    gc.collect()

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
..\torch\csrc\utils\python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


weird shape error



weird shape error



# tRAINING dTA

In [3]:
model_list = [
 'GWD_EPOCH_16_SCORE_0.0000_LOSS_203.7507.pt',
 'GWD_EPOCH_17_SCORE_0.0000_LOSS_201.9021.pt',
 'GWD_EPOCH_18_SCORE_0.0000_LOSS_203.8619.pt',
 'GWD_EPOCH_19_SCORE_0.0000_LOSS_200.8293.pt',
 'GWD_EPOCH_20_SCORE_0.0000_LOSS_197.1215.pt',
 'GWD_EPOCH_21_SCORE_0.0000_LOSS_195.4317.pt',
 'GWD_EPOCH_22_SCORE_0.0000_LOSS_194.9353.pt',
 'GWD_EPOCH_23_SCORE_0.0000_LOSS_193.9966.pt',
 'GWD_EPOCH_24_SCORE_0.0000_LOSS_193.6854.pt',
 'GWD_EPOCH_25_SCORE_0.0000_LOSS_194.8778.pt',
 'GWD_EPOCH_26_SCORE_0.0000_LOSS_193.3428.pt',
 'GWD_EPOCH_27_SCORE_0.0000_LOSS_194.3706.pt',
 'GWD_EPOCH_28_SCORE_0.0000_LOSS_192.9131.pt',
 'GWD_EPOCH_29_SCORE_0.0000_LOSS_191.8370.pt',
 'GWD_EPOCH_30_SCORE_0.0000_LOSS_190.1350.pt',
 'GWD_EPOCH_31_SCORE_0.0000_LOSS_191.7623.pt',
 'GWD_EPOCH_32_SCORE_0.0000_LOSS_189.1150.pt',
 'GWD_EPOCH_33_SCORE_0.0000_LOSS_189.5725.pt',
 'GWD_EPOCH_34_SCORE_0.0000_LOSS_188.9933.pt',
 'GWD_EPOCH_35_SCORE_0.0000_LOSS_190.0600.pt']

In [ ]:
torch.cuda.empty_cache()
for i in tqdm_notebook(range(len(model_list))):
    
    EPOCH = int(model_list[i].split('_')[2])
    LOSS = float(model_list[i].split('_')[-1][:-3])
    
    GWD_Model = torch.load('model/'+model_list[i])
    GWD_Model.eval()
    GWD_Model.to(conf.DEVICE)
    precisions = []
    
    for images, targets in tqdm_notebook(WD_Train_Loader):
        try:
            images = [torch.tensor(image, dtype = torch.float32).to(conf.DEVICE) for image in images]
            preds = GWD_Model(images)

            for pred, target in zip(preds, targets):
                ground_truth = target['boxes']
                pred_boxes = pred['boxes'].to(torch.device('cpu')).detach().numpy()
                precisions.append(calculate_image_precision(ground_truth, pred_boxes, thresholds=conf.PRECISION_THRESH))
        except:
            print('weird shape error')
    
    train_score = np.mean(np.array(precisions))
    #writer.add_scalar('Epoch_Vars/Model_Loss', LOSS, EPOCH)
    writer.add_scalar('Epoch_Vars/Training_Score', train_score, EPOCH)
    
    GWD_Model=None
    torch.cuda.empty_cache()
    gc.collect()

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
..\torch\csrc\utils\python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


weird shape error
weird shape error
weird shape error
weird shape error



weird shape error
weird shape error
weird shape error
weird shape error
weird shape error
weird shape error
weird shape error



weird shape error
weird shape error
weird shape error
weird shape error



weird shape error
weird shape error
weird shape error
weird shape error



...

weird shape error
weird shape error



weird shape error
weird shape error



weird shape error
weird shape error
weird shape error



weird shape error
weird shape error
weird shape error
weird shape error
weird shape error



weird shape error
weird shape error
weird shape error
weird shape error
weird shape error



In [39]:
i = [173, 3687, 4412, 113947, 117344, 118211, 121633, 121634, 147504, 147552]
f = pd.read_csv(conf.DATA_PATH+'/train.csv')
I = f[f.image_id == '893938464']
B = I[I.bbox=='[116.0, 177.0, 445.0, 133.0]']
B

,image_id,width,height,bbox,source
147552,893938464,1024,1024,"[116.0, 177.0, 445.0, 133.0]",arvalis_2


In [38]:
f.shape

(147793, 5)

In [40]:
I = f.drop(f.index[i])
I.shape

(147783, 5)